In [34]:
from pathlib import Path
import numpy as np
import pandas as pd
import random
import copy
import ast
import pprint
import json
import re

In [2]:
devices = ["cover", "switch", "light", "input", "temperature"]
devices = [device.capitalize() for device in devices]

# seeds generation

In [13]:
target_device = devices[2]

with open('prompts/prompt.md', "w") as f:
    f.write(f'')

with open('prompts/instruction_onedevice.md', "r") as f:
    instruction = f.read()
with open('prompts/seeds_multiapi.md', "r") as f:
    seeds = f.read()

with open('prompts/prompt.md', "a") as f:
    f.write(f'{instruction}\n\n'
            f'Example:\n\n{seeds}\n\n'
            f'For generation: \n\n'
            f'Device: {target_device} id=XXX\n\n')

row_i = 0
for p in Path('docs/').iterdir():
    if target_device in p.name and p.name != f'{target_device}.md':
        with open(p, "r") as f:
            method_str = f.read()
        with open('prompts/prompt.md', "a") as f:
            f.write(f'API method {row_i}: {method_str}\n')
        row_i += 1
    if row_i == 5:
        break

# with open('prompts/prompt.md', "a") as f:
#     f.write(f'User command 0:')

In [49]:
output = """[{"User command 0.1": "Reset the energy counter for the main light.", "JSON command 0.1": {"method": "Light.ResetCounters", "params": {"id": "id", "type": ["energy"]}}}, {"User command 0.2": "Reset the all counter for the light, please.", "JSON command 0.2": {"method": "Light.ResetCounters", "params": {"id": "id"}}}, {"User command 1.1" : "Could you please fetch the configuration details for the living room light, including its name, night mode settings, and transition duration?", "JSON command 1.1": {"method": "Light.GetConfig", "params": {"id": "id"}}}, {"User command 1.2" : "Tell me, what is transition duration for the main light?", "JSON command 1.2": {"method": "Light.GetConfig", "params": {"id": "id"}}}]"""
output = output.replace("XXX", '"id"')

In [50]:
output = re.sub(r'(User command \d\.\d)', 'User command', output)
output = re.sub(r'(JSON command \d\.\d)', 'JSON command', output)

In [51]:
print(output)

[{"User command": "Reset the energy counter for the main light.", "JSON command": {"method": "Light.ResetCounters", "params": {"id": "id", "type": ["energy"]}}}, {"User command": "Reset the all counter for the light, please.", "JSON command": {"method": "Light.ResetCounters", "params": {"id": "id"}}}, {"User command" : "Could you please fetch the configuration details for the living room light, including its name, night mode settings, and transition duration?", "JSON command": {"method": "Light.GetConfig", "params": {"id": "id"}}}, {"User command" : "Tell me, what is transition duration for the main light?", "JSON command": {"method": "Light.GetConfig", "params": {"id": "id"}}}]


In [52]:
import json

In [53]:
cmds = json.loads(output)

In [54]:
json.dumps(cmds)

'[{"User command": "Reset the energy counter for the main light.", "JSON command": {"method": "Light.ResetCounters", "params": {"id": "id", "type": ["energy"]}}}, {"User command": "Reset the all counter for the light, please.", "JSON command": {"method": "Light.ResetCounters", "params": {"id": "id"}}}, {"User command": "Could you please fetch the configuration details for the living room light, including its name, night mode settings, and transition duration?", "JSON command": {"method": "Light.GetConfig", "params": {"id": "id"}}}, {"User command": "Tell me, what is transition duration for the main light?", "JSON command": {"method": "Light.GetConfig", "params": {"id": "id"}}}]'

In [55]:
df_dict = {'device': [], 'user_cmd': [], 'used_mtd': [], 'json_cmd': []}
for row_i in range(len(cmds)):
    df_dict['user_cmd'].append(cmds[row_i][f'User command'])
    df_dict['json_cmd'].append(cmds[row_i][f'JSON command'])
    df_dict['used_mtd'].append(f'{cmds[row_i][f"JSON command"]["method"]}')
    df_dict['device'].append(target_device)
print(df_dict)

{'device': ['Light', 'Light', 'Light', 'Light'], 'user_cmd': ['Reset the energy counter for the main light.', 'Reset the all counter for the light, please.', 'Could you please fetch the configuration details for the living room light, including its name, night mode settings, and transition duration?', 'Tell me, what is transition duration for the main light?'], 'used_mtd': ['Light.ResetCounters', 'Light.ResetCounters', 'Light.GetConfig', 'Light.GetConfig'], 'json_cmd': [{'method': 'Light.ResetCounters', 'params': {'id': 'id', 'type': ['energy']}}, {'method': 'Light.ResetCounters', 'params': {'id': 'id'}}, {'method': 'Light.GetConfig', 'params': {'id': 'id'}}, {'method': 'Light.GetConfig', 'params': {'id': 'id'}}]}


In [56]:
data_path = Path('data/')

In [57]:

df = pd.DataFrame(df_dict)
df.to_csv(data_path / f'prompt_seeds.csv', index=False, mode='a', header=False)

# prompt generation

In [5]:
import pprint

In [13]:
device_methods_dict = {device: {'basic': [], 'advance': []} for device in devices}
advance_methods = ['GetConfig', 'SetConfig', 'GetStatus']
for p in Path('data/docs/methods').iterdir():
    for d in devices:
        if d in p.name:
            if p.stem.split('.')[1] in advance_methods:
                device_methods_dict[d]['advance'].append(p.stem)
            else:
                device_methods_dict[d]['basic'].append(p.stem)         
pprint.pprint(device_methods_dict)

{'Cover': {'advance': ['Cover.GetConfig', 'Cover.SetConfig', 'Cover.GetStatus'],
           'basic': ['Cover.Stop',
                     'Cover.Calibrate',
                     'Cover.GoToPosition',
                     'Cover.ResetCounters',
                     'Cover.Open',
                     'Cover.Close']},
 'Input': {'advance': ['Input.GetConfig', 'Input.SetConfig', 'Input.GetStatus'],
           'basic': ['Input.CheckExpression', 'Input.ResetCounters']},
 'Light': {'advance': ['Light.GetConfig', 'Light.GetStatus', 'Light.SetConfig'],
           'basic': ['Light.ResetCounters',
                     'Light.Set',
                     'Light.Toggle',
                     'Light.Calibrate']},
 'Switch': {'advance': ['Switch.SetConfig',
                        'Switch.GetStatus',
                        'Switch.GetConfig'],
            'basic': ['Switch.Toggle', 'Switch.ResetCounters', 'Switch.Set']},
 'Temperature': {'advance': ['Temperature.GetConfig',
                            

In [30]:
def generate_seed(device, seeds_df, num_seeds=2):
    seed_methods = random.sample(list(seeds_df[seeds_df['device'] == device]['used_mtd'].unique()), num_seeds)
    methods_description = ''
    cmds = []
    for method_i, method in enumerate(seed_methods):
        with open(f'data/docs/methods/{method}.md', "r") as f:
            method_str = f.read()
        methods_description += f'API method {method_i+1}: {method_str}\n'
        seed_ids = random.sample(list(seeds_df[seeds_df['used_mtd'] == method].index), 2)
        for seed_i, row_i in enumerate(seed_ids):
            cmds.append(
                {f'User command {method_i+1}.{seed_i+1}': seeds_df.loc[row_i, 'user_cmd'],
                 f'JSON command {method_i+1}.{seed_i+1}': ast.literal_eval(seeds_df.loc[row_i, 'json_cmd'].replace("'", '"'))}
            )
    seed_description = (
        f'The next device, methods and JSON are example:\n\n'
        f'Device: {device} id=33\n\n'
        f'{methods_description}'
        f'{json.dumps(cmds)}'
    )
    return seed_description

def generate_advance_seed(device, seeds_df):
    method = random.choice(list(seeds_df[seeds_df['device'] == device]['used_mtd'].unique()))
    cmds = []
    with open(f'data/docs/methods/{method}.md', "r") as f:
        method_str = f.read()
    method_description = f'API method 1:\n{method_str}\n'
    sample_methods_ids = list(seeds_df[seeds_df['used_mtd'] == method].index)
    seed_ids = random.sample(sample_methods_ids, min(4, len(sample_methods_ids)))
    for seed_i, row_i in enumerate(seed_ids):
        cmds.append(
            {f'User command {seed_i+1}': seeds_df.loc[row_i, 'user_cmd'],
            f'JSON command {seed_i+1}': ast.literal_eval(seeds_df.loc[row_i, 'json_cmd'].replace("'", '"'))}
        )
    seed_description = (
        f'The next device, methods and JSON are example:\n\n'
        f'Device: {device} id=33\n\n'
        f'{method_description}'
        f'{json.dumps(cmds)}'
    )
    return seed_description

In [31]:
seeds_df = pd.read_csv('data/prompt_seeds.csv')
# json.loads(seeds_df.loc[0, 'json_cmd'].replace("'", '"'))['method']
# f"dict: {json.dumps(ast.literal_eval(seeds_df.loc[0, 'json_cmd']))}"
print(generate_seed('Light', seeds_df))

The next device, methods and JSON are example:

Device: Light id=33

API method 1: Method name: Light.ResetCounters
Method description: This method resets associated counters (if applicable).
Request
Parameters:
{"id": {"type": "number", "description": "Id of the Light component instance. Required"}, "type": {"type": "array of strings", "description": "Array of strings, selects which counter to reset Optional"}}
Note. If no 'type' is provided, the method will reset all available counters.
Response
Attributes in the result:
{"aenergy": {"type": "object", "description": "Information about the active energy counter prior to reset", "properties": "{\"total\": {\"type\": \"number\", \"description\": \"Last counter value of the total energy consumed in Watt-hours\"}}"}}

API method 2: Method name: Light.Set
Method description: This method sets the output and brightness level of the Light component. It can be used to trigger webhooks. More information about the events triggering webhooks avai

In [68]:
NUM_METHODS_GEN = 2
NUM_METHODS_SEED = 2
EXAMPLES_BASIC_METHOD = 2
EXAMPLES_ADVANCE_METHOD = 8

In [69]:
seeds_df = pd.read_csv('data/prompt_seeds.csv')

for device, methods in device_methods_dict.items():
    method_i = 0
    methods_description = ''

    for i, method_name in enumerate(methods['basic']):
        with open(f'data/docs/methods/{method_name}.md', "r") as f:
            method_str = f.read()
        methods_description += f'API method {method_i+1}: {method_str}\n'
        method_i += 1

        if (method_i % NUM_METHODS_GEN == 0 and method_i != 0) or i == len(methods) - 1:
            with open('data/prompts/components/instruction_onedevice.md', "r") as f:
                instruction = f.read()
            instruction = instruction.replace('{NUM_EXAMPLE_COMMANDS}', str(method_i*EXAMPLES_BASIC_METHOD))

            seed_1 = generate_seed('Light', seeds_df, NUM_METHODS_SEED)
            # seed_description_2 = generate_seed('Light', seeds_df, 1)
            
            with open(f'data/prompts/prompt_basic_{device}_{i}.md', "w") as f:
                f.write(f'{instruction}\n\n'
                        f'{seed_1}\n\n'
                        # f'{seed_description_2}\n\n'
                        f'The next device and methods are for you to generate commands:\n\n'
                        f'Device: {device} id=XXX\n\n'
                        f'{methods_description}')
                
            method_i = 0
            methods_description = ''
    
    for i, method_name in enumerate(methods['advance']):
        with open(f'data/docs/methods/{method_name}.md', "r") as f:
            method_str = f.read()
        method_description = f'API method 1:\n{method_str}\n'

        with open('data/prompts/components/instruction_advance.md', "r") as f:
            instruction = f.read()
        instruction = instruction.replace('{NUM_EXAMPLE_COMMANDS}', str(EXAMPLES_ADVANCE_METHOD))

        seed_1 = generate_advance_seed('Light', seeds_df)
        
        with open(f'data/prompts/prompt_adv_{device}_{i}.md', "w") as f:
            f.write(f'{instruction}\n\n'
                    f'{seed_1}\n\n'
                    f'The next device and method are for you to generate commands:\n\n'
                    f'Device: {device} id=XXX\n\n'
                    f'{method_description}')

In [82]:
output = """[{"User command 1.1": "Stop the operation of the main cover immediately.", "JSON command 1.1": {"method": "Cover.Stop", "params": {"id": XXX}}}, {"User command 1.2": "Stop the kitchen cover now.", "JSON command 1.2": {"method": "Cover.Stop", "params": {"id": XXX}}}, {"User command 2.1": "Initiate the calibration procedure for the bedroom cover.", "JSON command 2.1": {"method": "Cover.Calibrate", "params": {"id": XXX}}}, {"User command 2.2": "Start the calibration process for the living room cover.", "JSON command 2.2": {"method": "Cover.Calibrate", "params": {"id": XXX}}}]"""
output = output.replace("XXX", '"id"')

In [83]:
output = re.sub(r'(User command \d*(\.\d*)?)', 'User command', output)
output = re.sub(r'(JSON command \d*(\.\d*)?)', 'JSON command', output)

In [84]:
print(output)

[{"User command": "Stop the operation of the main cover immediately.", "JSON command": {"method": "Cover.Stop", "params": {"id": "id"}}}, {"User command": "Stop the kitchen cover now.", "JSON command": {"method": "Cover.Stop", "params": {"id": "id"}}}, {"User command": "Initiate the calibration procedure for the bedroom cover.", "JSON command": {"method": "Cover.Calibrate", "params": {"id": "id"}}}, {"User command": "Start the calibration process for the living room cover.", "JSON command": {"method": "Cover.Calibrate", "params": {"id": "id"}}}]


In [85]:
cmds = json.loads(output)

In [86]:
df_dict = {'device': [], 'user_cmd': [], 'used_mtd': [], 'json_cmd': []}
for row_i in range(len(cmds)):
    df_dict['user_cmd'].append(cmds[row_i][f'User command'])
    df_dict['json_cmd'].append(cmds[row_i][f'JSON command'])
    df_dict['used_mtd'].append(f'{cmds[row_i][f"JSON command"]["method"]}')
    df_dict['device'].append(f'{cmds[row_i][f"JSON command"]["method"].split(".")[0]}')
print(df_dict)

{'device': ['Cover', 'Cover', 'Cover', 'Cover'], 'user_cmd': ['Stop the operation of the main cover immediately.', 'Stop the kitchen cover now.', 'Initiate the calibration procedure for the bedroom cover.', 'Start the calibration process for the living room cover.'], 'used_mtd': ['Cover.Stop', 'Cover.Stop', 'Cover.Calibrate', 'Cover.Calibrate'], 'json_cmd': [{'method': 'Cover.Stop', 'params': {'id': 'id'}}, {'method': 'Cover.Stop', 'params': {'id': 'id'}}, {'method': 'Cover.Calibrate', 'params': {'id': 'id'}}, {'method': 'Cover.Calibrate', 'params': {'id': 'id'}}]}


In [87]:
df = pd.DataFrame(df_dict)
df.to_csv('data/temp.csv', index=False, mode='a', header=False)